In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import datetime
import seaborn as sns

In [3]:
full_data = pd.read_csv('full_data.csv')
full_data.date = pd.to_datetime(full_data.date)
cols = ['new_cases', 'new_deaths', 'total_cases', 'total_deaths']
full_data[cols] = full_data[cols].apply(pd.to_numeric)

FileNotFoundError: [Errno 2] File b'full_data.csv' does not exist: b'full_data.csv'

In [ ]:
nl_data = full_data[full_data.location=='Netherlands']
it_data = full_data[full_data.location=='Italy']
#up to date
nl_data.head(100)

## Compare The Netherlands Vs Italy

In [ ]:
start_italy = it_data[it_data.new_cases!=0].date.iloc[0]
start_nl = nl_data[nl_data.new_cases!=0].date.iloc[0]

today = datetime.datetime.today()
range_dates_it = pd.date_range(start_italy,today).to_pydatetime().tolist()
range_dates_nl = pd.date_range(start_nl,today).to_pydatetime().tolist()
#it's possible to do the same for other countries

In [ ]:
fig = plt.figure(figsize=(15,4))
plt.plot(range_dates_it[1:], it_data.loc[it_data.date.isin(range_dates_it)].total_cases,lw=2,label='Italy')
plt.plot(range_dates_nl[1:], nl_data.loc[nl_data.date.isin(range_dates_nl)].total_cases,lw=2,label='The Netherlands')
plt.title('Total cases in two different countries')
plt.legend()
plt.show()

### Compute growth for death and total cases
##### Are the measures against coronavirus effective ?

In [ ]:
def perc_growth(sequence):
    return [round(100 * (b - a) / a,2) if a !=0 and b!=0 else 0 for a, b in zip(sequence[:], sequence[1:])]

In [ ]:
it_perc_growth = it_data[it_data.total_cases!=0][cols].apply(perc_growth)
nl_perc_growth = nl_data[nl_data.total_cases!=0][cols].apply(perc_growth)

fig,(ax1,ax2) = plt.subplots(nrows=1,ncols=2,figsize=(30,14))
fig.suptitle('Percentual Growth Italy vs Netherlands')

ax1.plot(range_dates_it[2:],it_perc_growth.total_cases,label='total cases');
ax1.plot(range_dates_it[2:],it_perc_growth.total_deaths,label='total deaths');
ax1.legend()
ax1.set(title='Italy',xlabel='date',ylabel='percentual growth %');


ax2.plot(range_dates_nl[2:],nl_perc_growth.total_cases,label='total cases');
ax2.plot(range_dates_nl[2:],nl_perc_growth.total_deaths,label='total deaths');
ax2.legend()
ax2.set(title='The Netherlands',xlabel='date',ylabel='percentual growth %');

#TODO: add notation on the government decision 

### Italian Region


In [ ]:
import os
os.getcwd()

In [ ]:
italy_regions = pd.read_csv('dpc-covid19-ita-regioni.csv')
italy_regions.tail(10)
df = italy_regions.dropna().sort_values(by=['denominazione_regione', 'data'])
df_lastday = df[df.data == df.data.max()]

df.head(5)

In [ ]:
tested = sns.barplot(data=df_lastday.sort_values(by='tamponi')[::-1],y='denominazione_regione',x='tamponi',color='darkblue',);
tested.set_title('Number of tested people for region')
tested.set_xlabel('Tests ( swabs )')
tested.set_ylabel('Region')
plt.show()

Number of ppl in Intensive-Cure units

In [ ]:
tested = sns.barplot(data=df_lastday.sort_values(by='terapia_intensiva')[::-1],y='denominazione_regione',x='terapia_intensiva',color='darkblue',);
tested.set_title('People in intensive-cure units per region')
tested.set_xlabel('number people')
tested.set_ylabel('Region')

In [ ]:
ax = df_lastday.sort_values(by='ricoverati_con_sintomi')[::-1][['denominazione_regione','ricoverati_con_sintomi','terapia_intensiva']].plot.bar(
    stacked=True,
    x='denominazione_regione',
    rot=15,
    figsize=(20,10))

ax.set(title='Total people hospitalized in Italy per region',xlabel='Region',ylabel='Number of people hospitalized',)
ax.legend(['Hospitalized with symptoms','Intensive-cure '])
plt.show()

In [ ]:
tested = sns.barplot(data=df_lastday.sort_values(by='ricoverati_con_sintomi')[::-1],y='denominazione_regione',x='ricoverati_con_sintomi',color='darkblue',);
tested.set_title('People Hospitalized')
tested.set_xlabel('number people')
tested.set_ylabel('Region')

In [ ]:
tested = sns.barplot(data=df_lastday.sort_values(by='deceduti')[::-1],y='denominazione_regione',x='deceduti',color='darkblue',);
tested.set_title('Deaths per region')
tested.set_xlabel('number people')
tested.set_ylabel('Region')

In [ ]:
#You can change the region
region_name = 'Lombardia'
region = df[df.denominazione_regione ==region_name]

fig = plt.figure(figsize=(10,10))

ax = sns.lineplot(x='data',y='totale_casi',data=region,label='total positive cases',lw=2)#
ax = sns.lineplot(x='data',y='tamponi',data=region,label='tested',lw=2)
ax.set_title('{} : number of people tested / positive to covid'.format(region_name))
ax.set_xticks(region.data.to_list()[::4])
ax.set_xlabel('Date')
ax.set_ylabel('Number of people')

#what's the percentage of ppl resulted positive with the test ? 
tot_tests = region[region.data == region.data.max()].tamponi.iloc[0]
tot_covid = region[region.data == region.data.max()].totale_casi.iloc[0]
#fig.text(0.9, 0.86,'Last day % of people positive after the test: {} %'.format(round(tot_covid/tot_tests*100)), fontsize=9,horizontalalignment='right',verticalalignment='top',)


#annotate on government policies
x_line_annotation = dt.datetime(2020,3,11)
ax.axvline(x=x_line_annotation, linestyle='dashed', alpha=0.3)
ax.text(x=x_line_annotation,y = tot_tests-10,s=' Initial Lockdown')

x_line_annotation = dt.datetime(2020,3,13)
ax.axvline(x=x_line_annotation, linestyle='dashed', alpha=0.3)
ax.text(x=x_line_annotation,y = tot_tests-0.1*tot_tests,s=' Only few activities open')


x_line_annotation = dt.datetime(2020,3,21)
ax.axvline(x=x_line_annotation, linestyle='dashed', alpha=0.3)
ax.text(x=dt.datetime(2020,3,14),y = 1000,s=' Last guidelines - total lockdown')


plt.show()

### Italy: tests vs covid-19

In [ ]:
italy_tot = df.groupby(by='data').agg('sum').reset_index()

fig = plt.figure(figsize=(10,10))

ax = sns.lineplot(x='data',y='totale_casi',data=italy_tot,label='total positive cases',lw=2)#
ax = sns.lineplot(x='data',y='tamponi',data=italy_tot,label='tested',lw=2)
ax.set_title('ITALY : number of people tested / positive to covid'.format(region_name))
ax.set_xticks(italy_tot.data.to_list()[::4])
ax.set_xlabel('Date')
ax.set_ylabel('Number of people')

#what's the percentage of ppl resulted positive with the test ? 
tot_tests = italy_tot[italy_tot.data == italy_tot.data.max()].tamponi.iloc[0]
tot_covid = italy_tot[italy_tot.data == italy_tot.data.max()].totale_casi.iloc[0]
#fig.text(0.9, 0.86,'Last day % of people positive after the test: {} %'.format(round(tot_covid/tot_tests*100)), fontsize=9,horizontalalignment='right',verticalalignment='top',)


#annotate on government policies
x_line_annotation = dt.datetime(2020,3,11)
ax.axvline(x=x_line_annotation, linestyle='dashed', alpha=0.3)
ax.text(x=x_line_annotation,y = tot_tests-10,s=' Initial Lockdown')

x_line_annotation = dt.datetime(2020,3,13)
ax.axvline(x=x_line_annotation, linestyle='dashed', alpha=0.3)
ax.text(x=x_line_annotation,y = tot_tests-0.1*tot_tests,s=' Only few activities open')


x_line_annotation = dt.datetime(2020,3,21)
ax.axvline(x=x_line_annotation, linestyle='dashed', alpha=0.3)
ax.text(x=dt.datetime(2020,3,14),y = 1000,s=' Last guidelines - total lockdown')


plt.show()

### New people with covid-19
Are the measures effective to reduce number of people infected ?

In [ ]:
italy_tot.head(5)

In [ ]:
italy_tot = df.groupby(by='data').agg('sum').reset_index()
nuovi_tamponi = [b - a for a,b in zip(italy_tot.tamponi,italy_tot.tamponi[1:])]
nuovi_tamponi.insert(0,np.nan)
italy_tot['nuovi_tamponi'] = nuovi_tamponi

fig = plt.figure(figsize=(10,10))

ax = sns.lineplot(x='data',y='nuovi_attualmente_positivi',data=italy_tot,label='new positive cases',lw=2)#
ax = sns.lineplot(x='data',y='nuovi_tamponi',data=italy_tot,label='tested',lw=2)
ax.set_title('ITALY : number of new tests  / daily positive to covid'.format(region_name))
ax.set_xticks(italy_tot.data.to_list()[::4])
ax.set_xlabel('Date')
ax.set_ylabel('Number of people')

#what's the percentage of ppl resulted positive with the test ? 
tot_tests = italy_tot[italy_tot.data == italy_tot.data.max()].nuovi_tamponi.iloc[0]
tot_covid = italy_tot[italy_tot.data == italy_tot.data.max()].totale_casi.iloc[0]
#fig.text(0.9, 0.86,'Last day % of people positive after the test: {} %'.format(round(tot_covid/tot_tests*100)), fontsize=9,horizontalalignment='right',verticalalignment='top',)


#annotate on government policies
x_line_annotation = dt.datetime(2020,3,11)
ax.axvline(x=x_line_annotation, linestyle='dashed', alpha=0.3)
ax.text(x=x_line_annotation,y = tot_tests-10,s=' Initial Lockdown')

x_line_annotation = dt.datetime(2020,3,13)
ax.axvline(x=x_line_annotation, linestyle='dashed', alpha=0.3)
ax.text(x=x_line_annotation,y = tot_tests-0.1*tot_tests,s=' Only few activities open')


x_line_annotation = dt.datetime(2020,3,21)
ax.axvline(x=x_line_annotation, linestyle='dashed', alpha=0.3)
ax.text(x=dt.datetime(2020,3,14),y = 1000,s=' Last guidelines - total lockdown')


plt.show()

### Total cases per region from the first infected patient registered

In [ ]:
tot_diff = (df.data.max() - df.data.min()).days

tot_cases, tot_deaths, tot_hospitalized, tot_icu = {},{},{},{}

for region in df.groupby(by=['denominazione_regione']):
    tot_cases[region[0]] = region[1].totale_casi.to_list()
    tot_deaths[region[0]] = region[1].deceduti.to_list()
    tot_hospitalized[region[0]] = region[1].totale_ospedalizzati.to_list()
    tot_icu[region[0]] = region[1].terapia_intensiva.to_list()
    
cum_tot_cases = pd.DataFrame(tot_cases)   
cum_tot_hospitalized = pd.DataFrame(tot_hospitalized)
cum_tot_icu = pd.DataFrame(tot_icu)   
cum_tot_deaths = pd.DataFrame(tot_deaths)

In [ ]:
fig, axes = plt.subplots(6,4,figsize=(20,15),constrained_layout=True)

regions = cum_tot_cases.columns.to_list()
k=0
for i in range(6):
    for j in range(4):
        sns.lineplot(y=regions[k],x= cum_tot_deaths.index.to_list(), data=cum_tot_deaths, ax=axes[i,j],color='r',label='tot_deaths')
        sns.lineplot(y=regions[k],x= cum_tot_hospitalized.index.to_list(),data = cum_tot_hospitalized,ax=axes[i,j],color='b',label='tot_hospitalized')
        sns.lineplot(y=regions[k],x= cum_tot_icu.index.to_list(),data = cum_tot_icu,ax=axes[i,j],color='g',label='tot_intensive_cure')
        
        sns.lineplot(y=regions[k],x= cum_tot_cases.index.to_list(),data = cum_tot_cases,ax=axes[i,j],color='k',label='tot_cases',alpha=.4)
        k=k+1
        if k == 21: break
fig.suptitle('How is the situation going in different regions?')
plt.show()


#### How 's the situation in the south  ? 
The goal is not to stress the healthcare system. There are a fixed number of intensive-cure units (ICU) in the hospitals. Overall, the healthcare in the south of Italy is worse compared to other regions in the north. Hence, keeping the number of people in ICU is essential to avoid unpleasent consequences


In [ ]:
ax = cum_tot_icu.plot(y=['Sicilia','Campania','Puglia','Calabria','Sicilia','Basilicata'],figsize=(10,10),kind='line')
ax.set(title='Intensive-cure in the South of Italy',xlabel='days after first case',ylabel='people in intensive-care unit')
plt.show()


df_cum